## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-11-23-53-38 +0000,nyt,U.S. Naval Academy Placed Under Lockdown Amid ...,https://www.nytimes.com/2025/09/11/us/naval-ac...
1,2025-09-11-23-50-59 +0000,nyt,False Tips and Chicken Coops: The Chaotic Hunt...,https://www.nytimes.com/2025/09/11/us/charlie-...
2,2025-09-11-23-49-31 +0000,bbc,Bolsonaro sentenced to 27 years in prison for ...,https://www.bbc.com/news/articles/c8xrqxk9p4xo...
3,2025-09-11-23-46-28 +0000,bbc,Nato strengthens defences after Russian drones...,https://www.bbc.com/news/articles/c0lkz2n34z6o...
4,2025-09-11-23-46-27 +0000,nyt,How An Obamacare Deadline Is Colliding With Sh...,https://www.nytimes.com/2025/09/11/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,kirk,71
14,charlie,64
44,trump,42
107,shooting,18
54,new,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
25,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...,239
336,2025-09-11-00-08-00 +0000,wsj,Charlie Kirk was a provocative conservative in...,https://www.wsj.com/politics/charlie-kirk-turn...,225
85,2025-09-11-21-16-12 +0000,nypost,Joe Rogan shocked after learning of Charlie Ki...,https://nypost.com/2025/09/11/us-news/joe-roga...,221
285,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...,220
9,2025-09-11-23-30-12 +0000,nypost,"Bluesky posters threaten Trump, Elon Musk, Ben...",https://nypost.com/2025/09/11/us-news/bluesky-...,209


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
25,239,2025-09-11-23-04-00 +0000,wsj,President Trump’s team began putting in place ...,https://www.wsj.com/politics/elections/angry-a...
285,60,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...
332,55,2025-09-11-00-22-00 +0000,wsj,Senate Republicans voted down a measure to rel...,https://www.wsj.com/politics/policy/senate-rep...
148,50,2025-09-11-17-58-01 +0000,latimes,New Paramount owner already has his sights on ...,https://www.latimes.com/entertainment-arts/bus...
226,50,2025-09-11-13-50-47 +0000,latimes,FBI releases photos of 'person of interest' in...,https://www.latimes.com/world-nation/story/202...
100,48,2025-09-11-20-18-21 +0000,bbc,Bolsonaro's coup trial gripped Brazil - and hi...,https://www.bbc.com/news/articles/cj3yekj2xygo...
289,42,2025-09-11-08-27-23 +0000,nypost,JD Vance pens heartfelt tribute to Charlie Kir...,https://nypost.com/2025/09/11/us-news/jd-vance...
11,42,2025-09-11-23-24-07 +0000,nypost,Hospital bigwig finally jailed for vicious att...,https://nypost.com/2025/09/11/us-news/hospital...
267,39,2025-09-11-10-00-00 +0000,latimes,California legislators strike last-minute deal...,https://www.latimes.com/california/story/2025-...
168,34,2025-09-11-17-00-00 +0000,wsj,Economists worry that a ‘universal basic incom...,https://www.wsj.com/economy/a-guaranteed-incom...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
